In [1]:
# https://jmetzen.github.io/2015-11-27/vae.html
# https://home.zhaw.ch/~dueo/bbs/files/vae.pdf

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)
tf.set_random_seed(0)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

n_samples = mnist.train.num_examples

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
input_shape = 784
n_hidden = 250
output_shape = 20 # 10-D Gaussian

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

W1_enc = weight_variable([input_shape, n_hidden])
b1_enc = bias_variable([n_hidden])
    
W2_enc_mean = weight_variable([n_hidden, output_shape/2])
b2_enc_mean = bias_variable([output_shape/2])

W2_enc_log_sd = weight_variable([n_hidden, output_shape/2])
b2_enc_log_sd = bias_variable([output_shape/2])
   
x = tf.placeholder(tf.float32, [None, 784])
x2 = tf.nn.softplus(tf.matmul(x, W1_enc) + b1_enc)
z_mean = tf.matmul(x2, W2_enc_mean) + b2_enc_mean # 5-D gaussian
z_log_sd2 = tf.matmul(x2, W2_enc_log_sd) + b2_enc_log_sd

eps = tf.random_normal(shape=[output_shape/2])

z = z_mean + tf.exp(z_log_sd2) * eps

W_dec = weight_variable([output_shape/2, n_hidden])
b_dec = bias_variable([n_hidden])

W1_dec = weight_variable([n_hidden, n_hidden])
b1_dec = bias_variable([n_hidden])
    
W2_dec = weight_variable([n_hidden, input_shape])
b2_dec = bias_variable([input_shape])

z1 = tf.nn.softplus(tf.matmul(z, W_dec) + b_dec)
z2 = tf.nn.softplus(tf.matmul(z1, W1_dec) + b1_dec)
x_hat = tf.nn.sigmoid(tf.matmul(z2, W2_dec) + b2_dec)

latent_loss = -0.5 * tf.reduce_sum(1 + z_log_sd2 - tf.square(z_mean) - tf.exp(z_log_sd2), 1)

reconstruction_loss = -tf.reduce_sum(x * tf.log(1e-10 + x_hat) + (1-x) * tf.log(1e-10 + 1 - x_hat), 1)

loss = tf.reduce_mean(latent_loss + reconstruction_loss)

In [5]:
batch_size = 100
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
training_epochs = 50
display_step = 5

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch = mnist.train.next_batch(batch_size)
            _, cost = sess.run([train_step, loss], feed_dict={x:  batch[0]})
            avg_cost += cost / n_samples * batch_size
            
        # Display logs per epoch step
        if epoch+1 % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), 
                  "cost=", "{:.9f}".format(avg_cost))

('Epoch:', '0001', 'cost=', '183.728795582')
('Epoch:', '0006', 'cost=', '111.212459689')
('Epoch:', '0011', 'cost=', '104.032178289')
('Epoch:', '0016', 'cost=', '100.183120367')
('Epoch:', '0021', 'cost=', '98.345527469')
('Epoch:', '0026', 'cost=', '96.809511691')
('Epoch:', '0031', 'cost=', '96.009689900')
('Epoch:', '0036', 'cost=', '95.097560466')
('Epoch:', '0041', 'cost=', '94.294942766')
('Epoch:', '0046', 'cost=', '93.962665683')
